In [26]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import requests as r
from Config2 import gkey
import json

city_data_df = pd.read_csv('../Weather Database/WeatherPy_Database.csv')
city_data_df.head()
city_data_df = city_data_df.set_index('City ID')

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,bambous virieux,MU,-20.3428,57.7575,74.21,88,75,3.44,broken clouds
1,1,bredasdorp,ZA,-34.5322,20.0403,60.12,82,0,2.30,clear sky
2,2,dingle,PH,10.9995,122.6711,83.26,76,88,8.14,overcast clouds
3,3,igrim,RU,63.1906,64.4162,-0.54,98,82,5.08,broken clouds
4,4,qaanaaq,GL,77.4840,-69.3632,-10.19,71,30,7.70,scattered clouds


In [2]:
min_temp = float(input('What is your minimum tolerable temperature?'))
max_temp = float(input('What is your maximum tolerable temperature?'))

What is your minimum tolerable temperature?60
What is your maximum tolerable temperature?90


In [3]:
preferred_temp_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)&
                                           (city_data_df['Max Temp'] <= max_temp)]


In [4]:
preferred_temp_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City ID,,,,,,,,,
0,bambous virieux,MU,-20.3428,57.7575,74.21,88,75,3.44,broken clouds
1,bredasdorp,ZA,-34.5322,20.0403,60.12,82,0,2.30,clear sky
2,dingle,PH,10.9995,122.6711,83.26,76,88,8.14,overcast clouds
6,saint-philippe,RE,-21.3585,55.7679,71.19,83,53,8.39,light rain
8,chabahar,IR,25.2919,60.6430,68.11,52,0,6.40,clear sky


In [5]:
preferred_temp_cities_df.isnull().any()

City                   False
Country                 True
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

In [6]:
preferred_temp_cities_df = preferred_temp_cities_df.dropna()
preferred_temp_cities_df.isnull().any()
preferred_temp_cities_df.count()

City                   False
Country                False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

City                   888
Country                888
Lat                    888
Lng                    888
Max Temp               888
Humidity               888
Cloudiness             888
Wind Speed             888
Current Description    888
dtype: int64

In [21]:
import gmaps
gmaps.configure(api_key=gkey)

In [92]:
hotel_df = preferred_temp_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng', 'Current Description']].copy()
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.drop_duplicates('City')
hotel_df
payload = {"radius": 5000, "type": "lodging", "key": gkey}
# base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
# hotel = r.get(base_url, params=payload).json()
# print(json.dumps(hotel, indent=4, sort_keys=True))


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
City ID,,,,,,,
0,bambous virieux,MU,74.21,-20.3428,57.7575,broken clouds,
1,bredasdorp,ZA,60.12,-34.5322,20.0403,clear sky,
2,dingle,PH,83.26,10.9995,122.6711,overcast clouds,
6,saint-philippe,RE,71.19,-21.3585,55.7679,light rain,
8,chabahar,IR,68.11,25.2919,60.6430,clear sky,
...,...,...,...,...,...,...,...
1749,mapiri,BO,68.59,-15.2500,-68.1667,overcast clouds,
1753,rio bueno,CL,66.34,-40.3167,-72.9667,clear sky,
1759,bella union,UY,69.44,-30.2752,-57.6007,overcast clouds,


In [93]:
for i, row in hotel_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    location = f'{lat},{lng}'
    payload['location'] = location
    hotel = r.get(base_url, params=payload).json()
    print(f'Getting data for {city}')
    try:
        hotel_df.loc[i,'Hotel Name'] = hotel['results'][0]['name']
    except IndexError:
        print('No hotel nearby, moving on...')
        pass
    

Getting data for bambous virieux
Getting data for bredasdorp
Getting data for dingle
Getting data for saint-philippe
Getting data for chabahar
Getting data for malanville
Getting data for cape town
Getting data for port elizabeth
Getting data for carnarvon
Getting data for tefe
Getting data for bandarbeyla
Getting data for souillac
Getting data for chuy
Getting data for aripuana
No hotel nearby, moving on...
Getting data for balsas
Getting data for hermanus
Getting data for port alfred
Getting data for victoria
Getting data for ferkessedougou
Getting data for itarema
Getting data for east london
Getting data for arraial do cabo
Getting data for bengkulu
Getting data for aquidauana
Getting data for san bernardo
Getting data for cockburn town
Getting data for mungwi
No hotel nearby, moving on...
Getting data for kruisfontein
Getting data for taltal
Getting data for georgetown
Getting data for luanda
Getting data for vargem alta
Getting data for cidreira
Getting data for marsh harbour
Get

Getting data for omboue
Getting data for matara
Getting data for lamu
Getting data for high rock
Getting data for port shepstone
Getting data for xique-xique
Getting data for cap malheureux
Getting data for luau
Getting data for arroio grande
Getting data for punta cardon
Getting data for nueve de julio
Getting data for bulungu
No hotel nearby, moving on...
Getting data for kinkala
Getting data for gisenyi
Getting data for port-gentil
Getting data for gwembe
No hotel nearby, moving on...
Getting data for cabedelo
Getting data for caxito
No hotel nearby, moving on...
Getting data for goma
Getting data for kudahuvadhoo
Getting data for sao gabriel da cachoeira
Getting data for ambilobe
Getting data for vredendal
Getting data for amboasary
Getting data for massakory
No hotel nearby, moving on...
Getting data for muscat
Getting data for nouakchott
Getting data for pitanga
Getting data for progreso
Getting data for franca
Getting data for doume
Getting data for doha
Getting data for abu dha

In [94]:
hotel_df
hotel_df['City'] = hotel_df['City'].str.capitalize()
hotel_df.isnull().any()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
City ID,,,,,,,
0,bambous virieux,MU,74.21,-20.3428,57.7575,broken clouds,Casa Tia Villa
1,bredasdorp,ZA,60.12,-34.5322,20.0403,clear sky,Bredasdorp Country Manor
2,dingle,PH,83.26,10.9995,122.6711,overcast clouds,SEGAYA PROPERTY
6,saint-philippe,RE,71.19,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
8,chabahar,IR,68.11,25.2919,60.6430,clear sky,هتل گدروشیا
...,...,...,...,...,...,...,...
1749,mapiri,BO,68.59,-15.2500,-68.1667,overcast clouds,
1753,rio bueno,CL,66.34,-40.3167,-72.9667,clear sky,Hostal Caulle
1759,bella union,UY,69.44,-30.2752,-57.6007,overcast clouds,Saville Free Shop


City                   False
Country                False
Max Temp               False
Lat                    False
Lng                    False
Current Description    False
Hotel Name             False
dtype: bool

In [100]:
#code refactored from https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python
hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(how='any', inplace=True)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
City ID,,,,,,,
0,Bambous virieux,MU,74.21,-20.3428,57.7575,broken clouds,Casa Tia Villa
1,Bredasdorp,ZA,60.12,-34.5322,20.0403,clear sky,Bredasdorp Country Manor
2,Dingle,PH,83.26,10.9995,122.6711,overcast clouds,SEGAYA PROPERTY
6,Saint-philippe,RE,71.19,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
8,Chabahar,IR,68.11,25.2919,60.6430,clear sky,هتل گدروشیا
...,...,...,...,...,...,...,...
377,Jalingo,NG,68.22,8.8833,11.3667,few clouds,Debright Motel
378,Bom jesus,BR,71.11,-9.0744,-44.3586,overcast clouds,Hotel Brasão & Floricultura Bella Flor
392,Bodden town,KY,82.00,19.2833,-81.2500,few clouds,Will T Place


In [103]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
City ID,,,,,,,
0,Bambous virieux,MU,74.21,-20.3428,57.7575,broken clouds,Casa Tia Villa
1,Bredasdorp,ZA,60.12,-34.5322,20.0403,clear sky,Bredasdorp Country Manor
2,Dingle,PH,83.26,10.9995,122.6711,overcast clouds,SEGAYA PROPERTY
6,Saint-philippe,RE,71.19,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
8,Chabahar,IR,68.11,25.2919,60.6430,clear sky,هتل گدروشیا
...,...,...,...,...,...,...,...
1745,Sechura,PE,72.03,-5.5569,-80.8222,broken clouds,Hospedaje Costa Bella
1753,Rio bueno,CL,66.34,-40.3167,-72.9667,clear sky,Hostal Caulle
1759,Bella union,UY,69.44,-30.2752,-57.6007,overcast clouds,Saville Free Shop


In [105]:
hotel_df.to_csv('WeatherPy_vacation.csv')

In [106]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
fig = gmaps.figure()
marker_locations = hotel_df[['Lat', 'Lng']]
hotel_info = [info_box_template.format(**row) for i, row in hotel_df.iterrows()]
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))